In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
from mrcnn.config import Config
from mrcnn import model as modellib, utils
import matplotlib.pyplot as plt
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg
from sklearn.model_selection import StratifiedKFold, KFold, train_test_split
import imgaug
import imgaug.augmenters as iaa
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
def get_ax(rows=1, cols=1, size=16):
  """Return a Matplotlib Axes array to be used in all visualizations in the notebook.  Provide a central point to control graph sizes. Adjust the size attribute to control how big to render images"""
  _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\framework\dtypes.py:5

In [1]:
Root_Dir=r"C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0" #main directory where all files are present
sys.path.append(Root_Dir)
COCO_Weights_path=os.path.join(Root_Dir,"mask_rcnn_coco.h5") # pretrained weights for mask rcnn used to initialize the network
Default_Logs_Dir=os.path.join(Root_Dir,"logs")

# config of mask rcnn, hyperparameters set here
class CustomConfig(Config):
    NAME="object"
    IMAGES_PER_GPU=1
    NUM_CLASSES=15+1
    BACKBONE ='resnet50'
    STEPS_PER_EPOCH=1475/(IMAGES_PER_GPU)
    VALIDATION_STEPS=336
    ROI_POSITIVE_RATIO = 0.334
    DETECTION_MIN_CONFIDENCE=0.3
    IMAGE_SHAPE=[1024,1024,3]
    IMAGE_MAX_DIM=1024
    IMAGE_MIN_DIM = 768
    IMAGE_RESIZE_MODE= "crop"
    LEARNING_RATE=0.001
    USE_MINI_MASK=False # AVOID DOWNSAMPLING to improve accuarcy. check if accuracy improves
    TRAIN_ROIS_PER_IMAGE=200 # REDUCE rois to improve runtime
    MAX_GT_INSTANCES=100 # reduce it from 100 to 50 to improve runtime
    RPN_ANCHOR_SCALES=(16, 32, 64, 128, 256)
    #POST_NMS_ROIS_INFERENCE=500 #halving the vaue to reduce runtime
    #POST_NMS_ROIS_TRAINING=1000 # having the value to reduce runtime
    #PRE_NMS_LIMIT=3000 # halving the value to reduce runtime
    
    
    
config=CustomConfig()
config.display()

NameError: name 'sys' is not defined

In [3]:
class InferenceConfig(Config):
    # Run detection on one image at a time
    NAME="object"
    IMAGES_PER_GPU=1
    BATCH_SIZE=1
    IMAGE_META_SIZE=28
    NUM_CLASSES=15+1
    BACKBONE ='resnet50'
    STEPS_PER_EPOCH=1475/(IMAGES_PER_GPU)
    VALIDATION_STEPS=336
    ROI_POSITIVE_RATIO = 0.334
    DETECTION_MIN_CONFIDENCE=0.3
    IMAGE_SHAPE=[1024,1024,3]
    IMAGE_MAX_DIM=1024
    LEARNING_RATE=0.001
    USE_MINI_MASK=False # AVOID DOWNSAMPLING to improve accuarcy. check if accuracy improves
    TRAIN_ROIS_PER_IMAGE=200 # REDUCE rois to improve runtime
    MAX_GT_INSTANCES=100 # reduce it from 100 to 50 to improve runtime
    RPN_ANCHOR_SCALES=(16, 32, 64, 128, 256)
    #POST_NMS_ROIS_INFERENCE=500 #halving the vaue to reduce runtime
    #POST_NMS_ROIS_TRAINING=1000 # having the value to reduce runtime
    #PRE_NMS_LIMIT=3000 # halving the value to reduce runtime
    

In [4]:
class CustomDataset(utils.Dataset):

    def load_custom(self, dataset_dir,file, subset):
        """Load a subset of the Dog-Cat dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have 32 classes to add.
        self.add_class("object",1,"Impaction")
        self.add_class("object",2,"Caries")
        self.add_class("object",3,"Shadow")
        self.add_class("object",4,"Bone loss")
        self.add_class("object",5,"Bridge")
        self.add_class("object",6,"Bdr")
        self.add_class("object",7,"Periapical Abscess")
        self.add_class("object",8,"Rct")
        self.add_class("object",9,"Crown")
        self.add_class("object",10,"Filling")
        self.add_class("object",11,"Unerupted")
        self.add_class("object",12,"Implant")
        self.add_class("object",13,"Rct +Post")
        self.add_class("object",14,"Splinted Crowns")
        self.add_class("object",15,"Retentive Pin")
        
        
        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        annotations1=json.load(open(file))['_via_img_metadata']
        # print(annotations1)
        annotations = list(annotations1.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]
        counter=0
        # Add images
        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']] 
            objects = [s['region_attributes']['Tooth'] for s in a['regions']]
            print("objects:",objects)
            name_dict = {"Impaction": 1,"Caries":2,"Shadow":3,"Bone Loss":4,"Bridge":5,"Bdr":6,"Periapical Abscess":7,"Rct":8,"Crown":9,"Filling":10,"Unerupted":11,"Implant":12,"Rct +Post":13,"Splinted Crowns":14,"Retentive Pin":15}
            
        
            # key = tuple(name_dict)
            num_ids = [name_dict[a] for a in objects]
     
            # num_ids = [int(n['Event']) for n in objects]
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            print("numids",num_ids)
            counter=counter+1
            print(counter)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",  ## for a single class just add the name here
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids
                )

    def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)
    def load_custom_K_fold(self, dataset_dir,file, subset, fold):
        # Add classes
        self.add_class("object",1,"Impaction")
        self.add_class("object",2,"Caries")
        self.add_class("object",3,"Shadow")
        self.add_class("object",4,"Bone Loss")
        self.add_class("object",5,"Bridge")
        self.add_class("object",6,"Bdr")
        self.add_class("object",7,"Periapical Abscess")
        self.add_class("object",8,"Rct")
        self.add_class("object",9,"Crown")
        self.add_class("object",10,"Filling")
        self.add_class("object",11,"Unerupted")
        self.add_class("object",12,"Implant")
        self.add_class("object",13,"Rct +Post")
        self.add_class("object",14,"Splinted Crowns")
        self.add_class("object",15,"Retentive Pin")
       

        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, 'train')        #split train folder to k-fold train and val

        N_Folds = 3
        
        annotations = []

        annotation =json.load(open(file))['_via_img_metadata']
        
        annotation = list(annotation.values())  # don't need the dict keys

        k_fold = KFold(n_splits = N_Folds, random_state = 42, shuffle = True)

        for i, (train, val) in enumerate(k_fold.split(annotation)):
            if subset == "train" and fold == i:
                for index in train:
                    annotations.append(annotation[index])

            elif subset == "val" and fold == i:
                for index in val:
                    annotations.append(annotation[index])

        annotations = [a for a in annotations if a['regions']]

        for a in annotations:
            # print(a)
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes (see json format above)
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['Tooth'] for s in a['regions']]
            name_dict = {"Impaction": 1,"Caries":2,"Shadow":3,"Bone Loss":4,"Bridge":5,"Bdr":6,"Periapical Abscess":7,"Rct":8,"Crown":9,"Filling":10,"Unerupted":11,"Implant":12,"Rct +Post":13,"Splinted Crowns":14,"Retentive Pin":15}
            num_ids = [name_dict[a] for a in objects]
            
            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            if os.path.exists(image_path):
                image = skimage.io.imread(image_path)
                height, width = image.shape[:2]

                self.add_image(
                    "object",  ## for a single class just add the name here
                    image_id=a['filename'],  # use file name as a unique image id
                    path=image_path,
                    width=width, height=height,
                    polygons=polygons,
                    num_ids=num_ids)

In [1]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=Default_Logs_Dir)

dataset_train = CustomDataset()
dataset_train.load_custom(r"C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\dataset",r"C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\dataset\train\train.json","train")
dataset_train.prepare()
dataset_val = CustomDataset()
dataset_val.load_custom(r"C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\dataset",r"C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\dataset\val\val.json", "val")
dataset_val.prepare()

model_inference = modellib.MaskRCNN(mode="inference", model_dir=Default_Logs_Dir, config=InferenceConfig) # prepare mask rcnn for testing
mAP_callback = modellib.MeanAveragePrecisionCallback(model, model_inference, dataset_val, 
                                                        calculate_at_every_X_epoch=1, dataset_limit=500, verbose=1)


NameError: name 'CustomConfig' is not defined

In [ ]:
weights_path = COCO_Weights_path
if not os.path.exists(weights_path):
    utils.download_trained_weights(weights_path)
       
model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230505T0210\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
1476/1476 [==============================] - 3013s 2s/step - loss: 1.9309 - rpn_class_loss: 0.0960 - rpn_bbox_loss: 0.6174 - mrcnn_class_loss: 0.3251 - mrcnn_bbox_loss: 0.4499 - mrcnn_mask_loss: 0.4424 - val_loss: 1.6303 - val_rpn_class_loss: 0.0638 - val_rpn_bbox_loss: 0.5790 - val_mrcnn_class_loss: 0.2903 - val_mrcnn_bbox_loss: 0.3572 - val_mrcnn_mask_loss: 0.3399
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230505T0210\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 34.44766776716317 %
mAR at epoch 1 is: 55.08028524339266 %
F1 score at epoch 1 is: 42.38647936832105 %
Epoch 2/15
1476/1476 [==============================] - 3005s 2s/step - loss: 1.6306 - rpn_class_loss: 0.0783 - rpn_bbox_loss: 0.5346 - mrcnn_class_loss: 0.2803 - mrcnn_bbox_loss: 0.3675 - mrcnn_mask_loss: 0.3700 - val_loss: 1.5236 - val_rpn_class_loss: 0.0562 - va

In [7]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=10,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230508T0154\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/10
1476/1476 [==============================] - 3028s 2s/step - loss: 1.2646 - rpn_class_loss: 0.0523 - rpn_bbox_loss: 0.4184 - mrcnn_class_loss: 0.2046 - mrcnn_bbox_loss: 0.2658 - mrcnn_mask_loss: 0.3234 - val_loss: 1.3314 - val_rpn_class_loss: 0.0478 - val_rpn_bbox_loss: 0.5002 - val_mrcnn_class_loss: 0.2353 - val_mrcnn_bbox_loss: 0.2740 - val_mrcnn_mask_loss: 0.2741
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230508T0154\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 57.95548256930354 %
mAR at epoch 1 is: 68.4731776192903 %
F1 score at epoch 1 is: 62.77684262508109 %
Epoch 2/10
1476/1476 [==============================] - 3011s 2s/step - loss: 1.2484 - rpn_class_loss: 0.0517 - rpn_bbox_loss: 0.4113 - mrcnn_class_loss: 0.2036 - mrcnn_bbox_loss: 0.2604 - mrcnn_mask_loss: 0.3215 - val_loss: 1.3095 - val_rpn_class_loss: 0.0458 - val

In [ ]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230509T0047\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
1475/1475 [==============================] - 3322s 2s/step - loss: 1.1268 - rpn_class_loss: 0.0464 - rpn_bbox_loss: 0.3703 - mrcnn_class_loss: 0.1823 - mrcnn_bbox_loss: 0.2220 - mrcnn_mask_loss: 0.3059 - val_loss: 1.2117 - val_rpn_class_loss: 0.0385 - val_rpn_bbox_loss: 0.4283 - val_mrcnn_class_loss: 0.2187 - val_mrcnn_bbox_loss: 0.2589 - val_mrcnn_mask_loss: 0.2673
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230509T0047\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 61.26415137630673 %
mAR at epoch 1 is: 71.71771883685933 %
F1 score at epoch 1 is: 66.0800630362882 %
Epoch 2/15
1475/1475 [==============================] - 3237s 2s/step - loss: 1.1290 - rpn_class_loss: 0.0447 - rpn_bbox_loss: 0.3739 - mrcnn_class_loss: 0.1847 - mrcnn_bbox_loss: 0.2200 - mrcnn_mask_loss: 0.3057 - val_loss: 1.2774 - val_rpn_class_loss: 0.0424 - val

In [ ]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230510T2352\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
1475/1475 [==============================] - 3231s 2s/step - loss: 1.0368 - rpn_class_loss: 0.0407 - rpn_bbox_loss: 0.3452 - mrcnn_class_loss: 0.1627 - mrcnn_bbox_loss: 0.1940 - mrcnn_mask_loss: 0.2942 - val_loss: 1.2316 - val_rpn_class_loss: 0.0365 - val_rpn_bbox_loss: 0.4619 - val_mrcnn_class_loss: 0.2053 - val_mrcnn_bbox_loss: 0.2581 - val_mrcnn_mask_loss: 0.2698
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230510T2352\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 62.37470061160755 %
mAR at epoch 1 is: 75.36743424103543 %
F1 score at epoch 1 is: 68.25828787507574 %
Epoch 2/15
1475/1475 [==============================] - 3202s 2s/step - loss: 1.0225 - rpn_class_loss: 0.0398 - rpn_bbox_loss: 0.3342 - mrcnn_class_loss: 0.1621 - mrcnn_bbox_loss: 0.1923 - mrcnn_mask_loss: 0.2941 - val_loss: 1.2251 - val_rpn_class_loss: 0.0364 - va

In [6]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')



Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230511T2340\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
1475/1475 [==============================] - 3630s 2s/step - loss: 0.9411 - rpn_class_loss: 0.0359 - rpn_bbox_loss: 0.3077 - mrcnn_class_loss: 0.1462 - mrcnn_bbox_loss: 0.1682 - mrcnn_mask_loss: 0.2830 - val_loss: 1.2229 - val_rpn_class_loss: 0.0367 - val_rpn_bbox_loss: 0.4462 - val_mrcnn_class_loss: 0.2178 - val_mrcnn_bbox_loss: 0.2589 - val_mrcnn_mask_loss: 0.2635
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230511T2340\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 65.37742725562514 %
mAR at epoch 1 is: 75.76761201343531 %
F1 score at epoch 1 is: 70.19009053939342 %
Epoch 2/15
1475/1475 [==============================] - 3366s 2s/step - loss: 0.9449 - rpn_class_loss: 0.0365 - rpn_bbox_loss: 0.3136 - mrcnn_class_loss: 0.1444 - mrcnn_bbox_loss: 0.1674 - mrcnn_mask_loss: 0.2829 - val_loss: 1.1891 - val_rpn_class_loss: 0.0343 - va

In [ ]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230514T2331\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
1475/1475 [==============================] - 3344s 2s/step - loss: 0.8682 - rpn_class_loss: 0.0341 - rpn_bbox_loss: 0.2888 - mrcnn_class_loss: 0.1283 - mrcnn_bbox_loss: 0.1469 - mrcnn_mask_loss: 0.2702 - val_loss: 1.2003 - val_rpn_class_loss: 0.0351 - val_rpn_bbox_loss: 0.4237 - val_mrcnn_class_loss: 0.2174 - val_mrcnn_bbox_loss: 0.2585 - val_mrcnn_mask_loss: 0.2656
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230514T2331\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 65.82785317878252 %
mAR at epoch 1 is: 77.79649988335038 %
F1 score at epoch 1 is: 71.31348497603193 %
Epoch 2/15
1475/1475 [==============================] - 3278s 2s/step - loss: 0.8677 - rpn_class_loss: 0.0334 - rpn_bbox_loss: 0.2891 - mrcnn_class_loss: 0.1264 - mrcnn_bbox_loss: 0.1475 - mrcnn_mask_loss: 0.2712 - val_loss: 1.1833 - val_rpn_class_loss: 0.0348 - va

In [6]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=15,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230516T0136\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/15
1475/1475 [==============================] - 3455s 2s/step - loss: 0.8188 - rpn_class_loss: 0.0325 - rpn_bbox_loss: 0.2696 - mrcnn_class_loss: 0.1189 - mrcnn_bbox_loss: 0.1355 - mrcnn_mask_loss: 0.2623 - val_loss: 1.2520 - val_rpn_class_loss: 0.0345 - val_rpn_bbox_loss: 0.4715 - val_mrcnn_class_loss: 0.2194 - val_mrcnn_bbox_loss: 0.2619 - val_mrcnn_mask_loss: 0.2648
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230516T0136\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 66.33154090398456 %
mAR at epoch 1 is: 78.17188749799543 %
F1 score at epoch 1 is: 71.7666260303609 %
Epoch 2/15
1475/1475 [==============================] - 3058s 2s/step - loss: 0.8155 - rpn_class_loss: 0.0321 - rpn_bbox_loss: 0.2731 - mrcnn_class_loss: 0.1146 - mrcnn_bbox_loss: 0.1344 - mrcnn_mask_loss: 0.2613 - val_loss: 1.1613 - val_rpn_class_loss: 0.0326 - val

KeyboardInterrupt: 

In [ ]:
Weights_path=os.path.join(Root_Dir,"temp.h5")
model.load_weights(Weights_path)
model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE/2,
                epochs=12,
                custom_callbacks=[mAP_callback],
                layers='heads')


Starting at epoch 0. LR=0.0005

Checkpoint Path: C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230517T0021\mask_rcnn_object_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
In model:  rpn_model
    rpn_conv_shared        (Conv2D)
    rpn_class_raw          (Conv2D)
    rpn_bbox_pred          (Conv2D)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask

C:\Users\HP\.conda\envs\mrcnn4\lib\site-packages\tensorflow\python\ops\gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/12
1475/1475 [==============================] - 3282s 2s/step - loss: 0.7794 - rpn_class_loss: 0.0299 - rpn_bbox_loss: 0.2611 - mrcnn_class_loss: 0.1077 - mrcnn_bbox_loss: 0.1254 - mrcnn_mask_loss: 0.2552 - val_loss: 1.2046 - val_rpn_class_loss: 0.0346 - val_rpn_bbox_loss: 0.4385 - val_mrcnn_class_loss: 0.2243 - val_mrcnn_bbox_loss: 0.2518 - val_mrcnn_mask_loss: 0.2555
Calculating mAP...
Loaded weights for the inference model (last checkpoint of the train model): C:\Users\HP\Desktop\dental\DentalPanoramicXrays\diagnosis_2.0\logs\object20230517T0021\mask_rcnn_object_0001.h5
Re-starting from epoch 1
mAP at epoch 1 is: 66.41944468846985 %
mAR at epoch 1 is: 76.67837024239785 %
F1 score at epoch 1 is: 71.18116756118764 %
Epoch 2/12
1475/1475 [==============================] - 3378s 2s/step - loss: 0.7633 - rpn_class_loss: 0.0302 - rpn_bbox_loss: 0.2504 - mrcnn_class_loss: 0.1053 - mrcnn_bbox_loss: 0.1238 - mrcnn_mask_loss: 0.2536 - val_loss: 1.2425 - val_rpn_class_loss: 0.0367 - va